In [1]:
!pip install -q bert-score python-Levenshtein evaluate rouge_score 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.9.1 which is incompatible.


In [2]:
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
from transformers import AutoProcessor, AutoModelForCausalLM
from bert_score import score as bertscore
from datasets import load_dataset
from tqdm import tqdm
import evaluate 
import Levenshtein

# Load test dataset using the datasets library
data_path = '/kaggle/input/luna-facad-custom/test_dataset-LUNA-FACAD.parquet'
test_dataset = load_dataset('parquet', data_files=data_path)['train']
test_images = test_dataset['image']
test_captions = test_dataset['text']

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
# Load evaluation metrics using the evaluate library
bleu = evaluate.load('bleu')
rouge = evaluate.load('rouge')
meteor = evaluate.load('meteor')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [4]:
# Initialize lists to store generated captions
blip_captions, blip_finetuned_captions, git_captions, git_finetuned_captions = [], [], [], []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Helper function to generate captions using a given model and processor
def generate_captions(model, processor, test_images):
    captions = []
    model = model.to(device)
    model.eval()  
    with torch.no_grad():
        for image in tqdm(test_images):
            inputs = processor(images=image, return_tensors="pt").to(device)
            pixel_values = inputs.pixel_values
            generated_ids = model.generate(pixel_values=pixel_values, max_length=64)
            caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
            captions.append(caption)
    return captions

In [5]:
# Load models and processors
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")

blip_finetuned_model = BlipForConditionalGeneration.from_pretrained("sagniksengupta/blip-finetuned-facad-v2")
blip_finetuned_processor = BlipProcessor.from_pretrained("sagniksengupta/blip-finetuned-facad-v2")

git_model = AutoModelForCausalLM.from_pretrained("microsoft/git-base")
git_processor = AutoProcessor.from_pretrained("microsoft/git-base")

git_finetuned_model = AutoModelForCausalLM.from_pretrained("sagniksengupta/git-finetuned-facad-v2")
git_finetuned_processor = AutoProcessor.from_pretrained("sagniksengupta/git-finetuned-facad-v2")

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/503 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [6]:
# Generate captions from all three models
blip_captions = generate_captions(blip_model, blip_processor, test_images)
blip_finetuned_captions = generate_captions(blip_finetuned_model, blip_finetuned_processor, test_images)
git_captions = generate_captions(git_model, git_processor, test_images)
git_finetuned_captions = generate_captions(git_finetuned_model, git_finetuned_processor, test_images)

100%|██████████| 2000/2000 [17:42<00:00,  1.88it/s]


In [21]:
import pandas as pd

df = pd.DataFrame({
    'image_id': [i for i in range(len(test_images))],
    'blip_caption': blip_captions,
    'blip_finetuned_caption': blip_finetuned_captions,
    'git_caption': git_captions,
    'git_finetuned_caption': git_finetuned_captions,
    'original_caption' : test_captions
})

df.to_csv('models_captions.csv', index=False)

In [22]:
df.head()

,image_id,blip_caption,blip_finetuned_caption,git_caption,git_finetuned_caption,original_caption
0,0,a woman in a bikini top and bottom,a tropical tropical print cover these soft str...,tropical palm tree bikini bottom in pink and g...,a bold plaid in a modern cut add to the laid b...,a ruched fold over waistband lends a retrro vi...
1,1,a woman wearing a black top and black pants,a soft and stretchy knit enhances the easygoin...,black dress with a long neckline,a sleek and sophisticated silhouette defines a...,long side slit allow flowy motion in this eleg...
2,2,a woman wearing a white t - shirt and blue jeans,a faded wash and faded stripe add vintage char...,person - i ' m in love with this!,a bold plaid in a modern cut add to the modern...,vintage inspired pinstripe extend the throwbac...
3,3,a woman wearing a yellow jacket and white pants,a vibrant floral print is the signature of thi...,silk silk kimono with a floral pattern.,a bold plaid in a modern cut add to the laid b...,veronica etro s global travel inform the kalei...
4,4,a woman wearing a black and gold dress,a halter neck neckline and a waist defining be...,black and white dress with a black bow in the ...,a sleek and sophisticated silhouette defines a...,lightweight crepe fabric give soft yet crisp s...


In [11]:
!pip install git+https://github.com/salaniz/pycocoevalcap

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Cloning https://github.com/salaniz/pycocoevalcap to /tmp/pip-req-build-t4_psuda
  Running command git clone --filter=blob:none --quiet https://github.com/salaniz/pycocoevalcap /tmp/pip-req-build-t4_psuda
  Resolved https://github.com/salaniz/pycocoevalcap to commit a24f74c408c918f1f4ec34e9514bc8a76ce41ffd
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.8/427.8 kB 6.6 MB/s eta 0:00:00a 0:00:01
  Created wheel for pycocoevalcap: filename=pycocoevalcap-1.2-py3-none-any.whl size=104312246 sha256=d53f12beb91bcc81861a52bd2d7fe1f9edb05578f42961f37eec4e5d83b93e96
  Stored in directory: /tmp/pip-ephem-wheel-cache-kj4uqyme/wheels/43/54/73/3e2c6d4ace7657958cde52ac6fd47b342cd4aae5a7aa4fcbf9
Successfully built pycocoevalcap


In [12]:
import evaluate
import Levenshtein
from bert_score import score as bertscore
from pycocoevalcap.cider.cider import Cider

# Load BLEU, ROUGE, METEOR metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")
cider_scorer = Cider()

def evaluate_generated_captions(generated_captions, test_captions):
    predictions = generated_captions
    references = [[ref] for ref in test_captions]  

    # Compute BLEU score
    bleu_scores = bleu.compute(predictions=predictions, references=references)['bleu']
    
    # Compute ROUGE score
    rouge_scores = rouge.compute(predictions=generated_captions, references=test_captions)
    
    # Compute BERTScore
    P, R, F1 = bertscore(generated_captions, test_captions, lang='en')
    bertscore_f1 = F1.mean().item()
    
    # Compute METEOR
    meteor_score = meteor.compute(predictions=generated_captions, references=test_captions)['meteor']
    
    # Compute CIDEr using pycocoevalcap
    cider_score, _ = cider_scorer.compute_score({i: [test_captions[i]] for i in range(len(test_captions))},
                                                {i: [generated_captions[i]] for i in range(len(generated_captions))})
    
    # Levenshtein distance calculation
    levenshtein_distances = []
    for gen_caption, ref_caption in zip(generated_captions, test_captions):
        distance = Levenshtein.distance(gen_caption, ref_caption)
        levenshtein_distances.append(distance)
    
    avg_levenshtein = sum(levenshtein_distances) / len(levenshtein_distances)

    return {
        'bleu': bleu_scores,
        'rouge': rouge_scores['rougeL'],
        'bertscore_f1': bertscore_f1,
        'meteor': meteor_score,
        'cider': cider_score,
        'avg_levenshtein': avg_levenshtein
    }


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [13]:
# Evaluate for each model's captions
blip_metrics = evaluate_generated_captions(blip_captions, test_captions)
blip_finetuned_metrics = evaluate_generated_captions(blip_finetuned_captions, test_captions)
git_metrics = evaluate_generated_captions(git_captions, test_captions)
git_finetuned_metrics = evaluate_generated_captions(git_finetuned_captions, test_captions)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

In [14]:
print("BLIP Base Metrics: ", blip_metrics)
print("BLIP Fine-tuned Metrics: ", blip_finetuned_metrics)
print("GIT Base Metrics: ", git_metrics)
print("GIT Fine-tuned Metrics: ", git_finetuned_metrics)

BLIP Base Metrics:  {'bleu': 0.0, 'rouge': 0.12953131437101992, 'bertscore_f1': 0.8314244151115417, 'meteor': 0.07409039076770353, 'cider': 0.0283613746907342, 'avg_levenshtein': 99.032}
BLIP Fine-tuned Metrics:  {'bleu': 0.029762342665750632, 'rouge': 0.19355394248821906, 'bertscore_f1': 0.8587872385978699, 'meteor': 0.17190936897690273, 'cider': 0.3147401795912754, 'avg_levenshtein': 94.496}
GIT Base Metrics:  {'bleu': 0.0, 'rouge': 0.10306075301954873, 'bertscore_f1': 0.831838846206665, 'meteor': 0.059931826324370795, 'cider': 0.02208585987285886, 'avg_levenshtein': 100.74}
GIT Fine-tuned Metrics:  {'bleu': 0.003863721661089574, 'rouge': 0.13013510797965544, 'bertscore_f1': 0.8465403914451599, 'meteor': 0.08927323377048495, 'cider': 0.05665661261245232, 'avg_levenshtein': 91.657}


In [15]:
import pandas as pd

data = [
    {'Model': 'BLIP Base', 'BLEU': blip_metrics['bleu'], 'ROUGE': blip_metrics['rouge'], 'BERTScore F1': blip_metrics['bertscore_f1'], 'Avg. Levenshtein': blip_metrics['avg_levenshtein'], 'METEOR': blip_metrics['meteor'], 'CIDEr': blip_metrics['cider']},
    {'Model': 'BLIP Fine-tuned', 'BLEU': blip_finetuned_metrics['bleu'], 'ROUGE': blip_finetuned_metrics['rouge'], 'BERTScore F1': blip_finetuned_metrics['bertscore_f1'], 'Avg. Levenshtein': blip_finetuned_metrics['avg_levenshtein'], 'METEOR': blip_finetuned_metrics['meteor'], 'CIDEr': blip_finetuned_metrics['cider']},
    {'Model': 'GIT Base', 'BLEU': git_metrics['bleu'], 'ROUGE': git_metrics['rouge'], 'BERTScore F1': git_metrics['bertscore_f1'], 'Avg. Levenshtein': git_metrics['avg_levenshtein'], 'METEOR': git_metrics['meteor'], 'CIDEr': git_metrics['cider']},
    {'Model': 'GIT Fine-tuned', 'BLEU': git_finetuned_metrics['bleu'], 'ROUGE': git_finetuned_metrics['rouge'], 'BERTScore F1': git_finetuned_metrics['bertscore_f1'], 'Avg. Levenshtein': git_finetuned_metrics['avg_levenshtein'], 'METEOR': git_finetuned_metrics['meteor'], 'CIDEr': git_finetuned_metrics['cider']}
]

df = pd.DataFrame(data)

df


,Model,BLEU,ROUGE,BERTScore F1,Avg. Levenshtein,METEOR,CIDEr
0,BLIP Base,0.000000,0.129531,0.831424,99.032,0.074090,0.028361
1,BLIP Fine-tuned,0.029762,0.193554,0.858787,94.496,0.171909,0.314740
2,GIT Base,0.000000,0.103061,0.831839,100.740,0.059932,0.022086
3,GIT Fine-tuned,0.003864,0.130135,0.846540,91.657,0.089273,0.056657


In [19]:
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration, VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer, CLIPProcessor, CLIPModel

# Load the CLIP model and processor, then move the model to the GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

# Define a function to compute CLIP logits for an image and list of captions
def get_clip_logits(image, captions):
    # Prepare inputs and move them to the appropriate device
    inputs = clip_processor(text=captions, images=image, return_tensors="pt", padding=True).to(device)
    outputs = clip_model(**inputs)
    logits_per_image = outputs.logits_per_image  # Image-text similarity scores
    return logits_per_image

# Define a function to compute CLIP logits for a batch of images and captions
def calculate_clip_logits_for_captions(image_list, caption_list):
    logits = []
    for image, caption in zip(image_list, caption_list):
        logits_per_image = get_clip_logits(image, [caption])
        logits.append(logits_per_image.item())  # Extract the scalar value for each image-caption pair
    return logits

# Generate CLIP logits for each model's captions
blip_clip_logits = calculate_clip_logits_for_captions(test_images, blip_captions)
blip_finetuned_clip_logits = calculate_clip_logits_for_captions(test_images, blip_finetuned_captions)
git_clip_logits = calculate_clip_logits_for_captions(test_images, git_captions)
git_finetuned_clip_logits = calculate_clip_logits_for_captions(test_images, git_finetuned_captions)


In [24]:
data = [
    {'Model': 'BLIP Base', 'BLEU': blip_metrics['bleu'], 'ROUGE': blip_metrics['rouge'], 'BERTScore F1': blip_metrics['bertscore_f1'], 'Avg. Levenshtein': blip_metrics['avg_levenshtein'], 'METEOR': blip_metrics['meteor'], 'CIDEr': blip_metrics['cider'], 'Avg. CLIP Logits': torch.mean(torch.tensor(blip_clip_logits)).item()},
    {'Model': 'BLIP Fine-tuned', 'BLEU': blip_finetuned_metrics['bleu'], 'ROUGE': blip_finetuned_metrics['rouge'], 'BERTScore F1': blip_finetuned_metrics['bertscore_f1'], 'Avg. Levenshtein': blip_finetuned_metrics['avg_levenshtein'], 'METEOR': blip_finetuned_metrics['meteor'], 'CIDEr': blip_finetuned_metrics['cider'], 'Avg. CLIP Logits': torch.mean(torch.tensor(blip_finetuned_clip_logits)).item()},
    {'Model': 'GIT Base', 'BLEU': git_metrics['bleu'], 'ROUGE': git_metrics['rouge'], 'BERTScore F1': git_metrics['bertscore_f1'], 'Avg. Levenshtein': git_metrics['avg_levenshtein'], 'METEOR': git_metrics['meteor'], 'CIDEr': git_metrics['cider'], 'Avg. CLIP Logits': torch.mean(torch.tensor(git_clip_logits)).item()},
    {'Model': 'GIT Fine-tuned', 'BLEU': git_finetuned_metrics['bleu'], 'ROUGE': git_finetuned_metrics['rouge'], 'BERTScore F1': git_finetuned_metrics['bertscore_f1'], 'Avg. Levenshtein': git_finetuned_metrics['avg_levenshtein'], 'METEOR': git_finetuned_metrics['meteor'], 'CIDEr': git_finetuned_metrics['cider'], 'Avg. CLIP Logits': torch.mean(torch.tensor(git_finetuned_clip_logits)).item()}
]

df = pd.DataFrame(data)

df

,Model,BLEU,ROUGE,BERTScore F1,Avg. Levenshtein,METEOR,CIDEr,Avg. CLIP Logits
0,BLIP Base,0.000000,0.129531,0.831424,99.032,0.074090,0.028361,23.352610
1,BLIP Fine-tuned,0.029762,0.193554,0.858787,94.496,0.171909,0.314740,25.839006
2,GIT Base,0.000000,0.103061,0.831839,100.740,0.059932,0.022086,24.227318
3,GIT Fine-tuned,0.003864,0.130135,0.846540,91.657,0.089273,0.056657,13.080656


In [25]:
df.to_csv('model_metrics.csv')